## Points to remember

Write down the points to be remembered in this cell while coding, so it might be helpful to others working on same code - different functions

 
FLAT -> 0 </br>
HILLY -> 1 </br>
FORREST -> 2 </br>
BLOCK -> 3 </br>

In [1]:
import numpy as np
import sympy as sym
import random
from scipy.spatial.distance import cityblock

In [7]:
class agent_9:

    def __init__(self, maze_size):
        self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
#         print("self.belief_matrix", self.belief_matrix)
        self.visited_matrix = np.matrix(np.zeros((maze_size, maze_size),dtype=bool))
        self.agent_gridworld = np.matrix(np.zeros((maze_size, maze_size)))
        self.agent_gridworld_0_1 = np.matrix(np.zeros((maze_size, maze_size)))
        self.certainty_matrix = np.full((maze_size, maze_size), 0.35/maze_size*maze_size)

    def getMaxProbabilityCell(self, source):
#         array_sum = np.sum(self.belief_matrix)
#         array_has_nan = np.isnan(array_sum)
#         if array_has_nan:
#             self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))

#         mat = self.belief_matrix
#         maxProb = mat.max()
        
        array_sum = np.sum(self.certainty_matrix)
        array_has_nan = np.isnan(array_sum)
        if array_has_nan:
            self.certainty_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))

        mat = self.certainty_matrix
        maxProb = mat.max()
        
        
        occurences = list(np.where(mat == maxProb))
        number_of_occurrences = len(occurences[0])
        node_to_return = source
        if number_of_occurrences > 1:
            dist = []
            previous_manhattan_distance = 9999
            for index in range(number_of_occurrences):
                
                _row = occurences[0][index]
                _col = occurences[1][index]

                target_node = (_row, _col)
                # find manhattan distance
                if target_node != source:
                    current_manhattan_distance = cityblock([source], target_node)
                    dist.append([target_node, current_manhattan_distance])

                    # code for finding manhattan distance between current node and the node in loop
                    if current_manhattan_distance < previous_manhattan_distance:
                        previous_manhattan_distance = current_manhattan_distance
#                     node_to_return = target_node
            dist = [x for x in dist if x[1]==previous_manhattan_distance]
            node_to_return = dist[np.random.randint(0,len(dist))][0]

        else:
            row = occurences[0][0]
            col = occurences[1][0]
            node_to_return = (row, col)
        print("The next target to be visited is", node_to_return)
        return node_to_return


    def examine(self, node, terrain_type, maze):
        if node ==  maze.target:
#             print("examine target")
            p = np.random.rand()
            if terrain_type == 0:
                if p >= 0.2:
                    return True
            elif terrain_type == 1:
                if p >= 0.5:
                    return True
            elif terrain_type == 2:
                if p >= 0.8:
                    return True
        return False

    def update_beliefs_when_xy_blocked(self, x, y, maze_size):
        for i in range(maze_size):
            for j in range(maze_size):
                if i==x and j==y:
                    continue
                self.belief_matrix[i][j] = self.belief_matrix[i][j]/(1 - self.belief_matrix[x][y])
        self.belief_matrix[x][y]=0
        
    
    def update_beliefs_when_xy_failed_with_some_terrainType(self, x, y, t, maze_size):

        sum_of_pij = 0
        for i in range(maze_size):
            for j in range(maze_size):
                if i==x and j==y:
                    continue
                sum_of_pij = sum_of_pij + self.belief_matrix[i][j]
        p_b=0
        if t==0:
            p_b=sum_of_pij+(0.2*self.belief_matrix[x][y])
            self.belief_matrix[x][y] = (self.belief_matrix[x][y]*0.2)/p_b
        elif t == 1:
            p_b=sum_of_pij+(0.5*self.belief_matrix[x][y])
            self.belief_matrix[x][y] = (self.belief_matrix[x][y]*0.5)/p_b
        elif t == 2:
            p_b=sum_of_pij+(0.8*self.belief_matrix[x][y])
            self.belief_matrix[x][y] = (self.belief_matrix[x][y]*0.8)/p_b


        for i in range(maze_size):
            for j in range(maze_size):
                if i==x and j==y:
                    continue
                self.belief_matrix[i][j] = self.belief_matrix[i][j]/p_b
#         print('self.belief_matrix', self.belief_matrix)


    def update_belief(self, maze, type_1, type_2):
        new_belief = self.belief_matrix
        # bel = self.belief_matrix

        terrain_indices_type1 = list(zip(*np.where(maze.terrain_matrix != type_1)))
        terrain_indices_type2 = list(zip(*np.where(maze.terrain_matrix != type_2)))
        # terrain_indices_blocked = list(zip(*np.where(maze.terrain_matrix != type_3)))

        other_type = [value for value in terrain_indices_type1 if value in terrain_indices_type2]
        # other_type = [value for value in d if value in terrain_indices_blocked]

        for i in other_type:
            new_belief.itemset(i,0)

        for (row, column) in terrain_indices_type2:
            # Iterate through the neighbours:
            type1_counts = 0
            surrounding_type1 = []
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if i == 0 and j == 0:
                        continue
                    if (row + i, column + j) in terrain_indices_type1:
                        type1_counts += 1
                        surrounding_type1.append((row + i, column + j))

                # Divide the current belief of type2 cell equally among the surrounding type1 cells
                if type1_counts != 0:
                    belief_delta = self.belief_matrix[row, column]/type1_counts
                    for (r, c) in surrounding_type1:
                        new_belief[r, c] += belief_delta

        # Update all type2 cells beside a type1 cell
        for (row, column) in terrain_indices_type1:
            # Iterate through the neighbours:
            type2_counts = 0
            surrounding_type2 = []
            for i in [-1, 0, 1]:
                for j in [-1, 0, 1]:
                    if i == 0 and j == 0:
                        continue
                    if (row + i, column + j) in terrain_indices_type2:
                        type2_counts += 1
                        surrounding_type2.append((row + i, column + j))

            # Divide the current belief of type1 cell equally among the surrounding type2 cells
            if type2_counts != 0:
                belief_delta = self.belief_matrix[row, column] / type2_counts
                for (r, c) in surrounding_type2:
                    new_belief[r, c] += belief_delta

        # Normalise the new belief matrix
                try:
                    new_belief /= new_belief.sum()
                    self.belief_matrix = new_belief
                except Exception as err:
                    print("Exception: ", err)
                    self.belief_matrix = np.full((maze_size, maze_size), 1/(maze_size*maze_size))
#         print('self.belief_matrix', self.belief_matrix)


    def update_certainties(self, maze):

        t_mat = maze.terrain_matrix
        bel_mat = self.belief_matrix
        v_mat = self.visited_matrix
        c_mat = self.certainty_matrix

        for i in range(maze_size):
            for j in range(maze_size):
                if v_mat.item(i,j):
                    if t_mat.item(i,j) == 0:
                        c_mat[i,j] = bel_mat[i,j]*0.8
                    if t_mat.item(i,j) == 1:
                        c_mat[i,j] = bel_mat[i,j]*0.5
                    if t_mat.item(i,j) == 2:
                        c_mat[i,j] = bel_mat[i,j]*0.2
                else:
                    c_mat[i,j] = bel_mat[i,j]*0.35

#         Sum of the confidence matrix
#         conf_sum = np.sum(c_mat)
#         print("conf_sum", conf_sum)
        # Normalize the confidence matrix
#         c_mat = c_mat
#         c_mat = c_mat/conf_sum

#         print('c_mat', c_mat)
#         print('self.certainty_matrix', self.certainty_matrix)
#         print(self.belief_matrix)

In [9]:
class gridworld:
    
    def __init__(self, maze_size):
#         self.full_grid_world_matrix = []
        self.terrain_matrix = self.initialize_terrain_matrix(maze_size)
        self.full_grid_world_matrix = self.initialize_grid_world(maze_size)
        self.source, self.target = self.set_valid_source_target(maze_size)
        self.maze_size = maze_size
#         self.terrain_types = ['flat', 'hilly', 'forest', 'blocked']
        
    def target_moves(self, maze_size , current_node, valid = []):
        type1 = self.terrain_matrix.item(self.target[0], self.target[1])
        possible_moves = [(0,1), (0,-1), (1,0), (-1,0), (-1,-1), (-1,1), (1,-1), (1,1)]
        valid = []
        choose = random.randint(1, len(possible_moves)) - 1
        target_row = maze.target[0] + possible_moves[choose][0]
        target_col = maze.target[0] + possible_moves[choose][1]
        if target_row > -1 and target_row < maze_size and target_col > -1 and target_col < maze_size:
            self.target = (target_row, target_col)
            type2 = self.terrain_matrix.item(self.target[0], self.target[1])
            # if old and new terrain types are same then, move target to another neighbouring node
            if type1 == type2:
                self.target_moves(maze_size, current_node)
        else:
            self.target_moves(maze_size, current_node)

        # if the target is blocked, move target to another neighbouring node
        if self.full_grid_world_matrix.item(self.target) == 1:
            self.target_moves(maze_size, current_node)

        type2 = self.terrain_matrix.item(self.target[0], self.target[1])
#         print("The Actual target is", self.target)
#         print("Actual Target moved from ", self.terrain_types[type1], " to ", self.terrain_types[type2])

        cells_that_might_have_target = list(zip(*np.where(maze.terrain_matrix == type2)))
        for node in cells_that_might_have_target:
            for neighbor in possible_moves:
                new_cell = (node[0] + neighbor[0], node[1] + neighbor[1])
                if new_cell[0] > -1 and new_cell[0] < maze_size and new_cell[1] > -1 and new_cell[1] < maze_size:
                    if self.terrain_matrix.item(new_cell) == type2:
                        valid.append(new_cell)
                        
#         print('valid', valid)
                    
        if valid:
            x = current_node[0]
            y = current_node[1]
            current_node_neighbors = []
            for i in possible_moves:
                neighbor = x + i[0] , y + i[1]
                current_node_neighbors.append(neighbor)
            probable_neighbors_with_target = [value for value in current_node_neighbors if value in valid]
            if probable_neighbors_with_target:
                choose = random.randint(1, len(probable_neighbors_with_target)) - 1
                max_prob_cell = probable_neighbors_with_target[choose]
            else:
                choose = random.randint(1, len(valid)) - 1
                #assume first valid cell to be max probability
                max_prob = self.terrain_matrix.item(valid[0])
                for cell in valid:
                    if max_prob >=  self.terrain_matrix.item(cell):
                        max_prob_cell = cell
    #             print('new_target to visit is', valid[choose])
            return max_prob_cell, type1, type2

    def initialize_terrain_matrix(self, maze_size):
        # generates number between 0-2
        np_array = np_array = np.random.rand(maze_size*maze_size)
        block = lambda x: 3 if x<0.3 else 0 # setting blocks
        vectorized_block = np.vectorize(block)
        np_array = vectorized_block(np_array)
#         print(np_array.reshape(maze_size,maze_size))
        self.full_grid_world_matrix = np_array  #setting full_gridworld_matrix
        
        terrain_probs = lambda x: 3 if x==3 else np.random.rand()
        vectorized_terrain_probs = np.vectorize(terrain_probs)
        np_array = vectorized_terrain_probs(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        set_terrain = lambda x: x if x==3 else(0 if x<=0.33 else (1 if x<=0.66 else 2))
        vectorized_set_terrain = np.vectorize(set_terrain)
        np_array = vectorized_set_terrain(np_array)
#         print(np_array.reshape(maze_size, maze_size))
        
        terrain_matrix = np.asmatrix(np_array).reshape(maze_size,maze_size)
#         print(terrain_matrix)
        return terrain_matrix

    def initialize_grid_world(self, maze_size):

        block = lambda x: 1 if x==3 else x
        vectorize_block = np.vectorize(block)
        self.full_grid_world_matrix = vectorize_block(self.full_grid_world_matrix)
        self.full_grid_world_matrix = np.asmatrix(self.full_grid_world_matrix).reshape(maze_size,maze_size)
    #     plt.spy(full_grid_world_matrix)
        return self.full_grid_world_matrix

    def set_valid_source_target(self, maze_size):

        target_row = random.randint(0, maze_size-1)
        target_col = random.randint(0, maze_size-1)

        source_row = random.randint(0, maze_size-1)
        source_col = random.randint(0, maze_size-1)

        while (self.full_grid_world_matrix.item(target_row, target_col) == 1):
            target_row = random.randint(0, maze_size-1)
            target_col = random.randint(0, maze_size-1)
            
        while(self.full_grid_world_matrix.item(source_row, source_col) == 1):
            source_row = random.randint(0, maze_size-1)
            source_col = random.randint(0, maze_size-1)
        
        source = (source_row, source_col)
        target = (target_row, target_col)

        return source, target
               

In [15]:
i = [1,2,3]   # create the list instance, and bind it to i
j = i         # bind j to the same list as i
i[0] = 5      # change the first item of i
j.append(4)
i

[5, 2, 3, 4]